In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **나이브 베이지안(Naive Bayesian) 알고리즘**

Corpus를 활용하여 추출


In [ ]:
import os
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from openpyxl import Workbook

# 문제 해결과 관련된 키워드 목록 정의
problem_solving_keywords = ['error', 'debugging', 'algorithm', 'optimization']

# 경로 설정
directory_path = '/content/drive/MyDrive/four programming/four programming_Q_list'

# 데이터 로드 및 레이블링
def load_and_label_questions(directory, keywords):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            language = filename.split(".txt")[0]  # 프로그래밍 언어 식별
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                for line in file:
                    line = line.strip().lower()
                    label = 'no'
                    for keyword in keywords:
                        if keyword in line:
                            label = keyword
                            break
                    data.append({"language": language, "question": line, "label": label})
    return pd.DataFrame(data)

# 나이브 베이지안 알고리즘을 사용하여 질문 분류
def classify_questions(data):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(data['question'])
    y = (data['label'] != 'no').astype(int)  # 문제 해결 관련 여부를 0, 1로 변환

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    classifier = MultinomialNB()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print(classification_report(y_test, y_pred))

    return classifier, vectorizer

# 엑셀 파일로 저장
def save_to_excel(data, filename='classified_questions.xlsx'):
    data.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

# 메인 실행 함수
def main():
    data = load_and_label_questions(directory_path, problem_solving_keywords)
    classifier, vectorizer = classify_questions(data)
    save_to_excel(data)

# 실행
if __name__ == "__main__":
    main()

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2767
           1       0.65      0.75      0.69       233

    accuracy                           0.95      3000
   macro avg       0.81      0.86      0.83      3000
weighted avg       0.95      0.95      0.95      3000

Data saved to classified_questions.xlsx


파일 4개를 한번에 묶어서 추출

In [ ]:
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# 문제 해결과 관련된 키워드 목록 정의
problem_solving_keywords = ['error', 'debugging', 'algorithm', 'optimization']

# 파일 목록
file_paths = [
    '/content/drive/MyDrive/four programming/four programming_Q_list/C_data_Q_list.txt',
    '/content/drive/MyDrive/four programming/four programming_Q_list/java_data_Q_list.txt',
    '/content/drive/MyDrive/four programming/four programming_Q_list/Python_data_Q_list.txt',
    '/content/drive/MyDrive/four programming/four programming_Q_list/Prolog_data_Q_list.txt'
]

# 파일 데이터 로드 및 레이블링
def load_and_label_questions(file_paths, keywords):
    data = []
    for filepath in file_paths:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                line = line.strip().lower()
                label = 'no'
                for keyword in keywords:
                    if keyword in line:
                        label = keyword
                        break
                data.append({"question": line, "label": label})
    return pd.DataFrame(data)

# 나이브 베이지안 알고리즘을 사용하여 질문 분류
def classify_questions(data):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(data['question'])
    y = (data['label'] != 'no').astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    classifier = MultinomialNB()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    return classifier, vectorizer

# 엑셀 파일로 저장
def save_to_excel(data, filename='four_data_classified_questions.xlsx'):
    data.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

# 메인 실행 함수
def main():
    data = load_and_label_questions(file_paths, problem_solving_keywords)
    classifier, vectorizer = classify_questions(data)
    save_to_excel(data)

# 실행
if __name__ == "__main__":
    main()

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2772
           1       0.67      0.74      0.70       228

    accuracy                           0.95      3000
   macro avg       0.82      0.85      0.84      3000
weighted avg       0.95      0.95      0.95      3000

Data saved to four_data_classified_questions.xlsx


성능 지표: 코드는 테스트 데이터에 대한 나이브 베이지안 분류기의 성능을 평가하여 정밀도(precision), 재현율(recall), F1 점수와 같은 지표를 제공합니다. 이러한 지표들은 모델이 얼마나 잘 문제 해결 질문을 식별하는지를 나타냅니다.

정밀도(Precision): 모델이 문제 해결 질문으로 분류한 것 중 실제로 문제 해결 질문인 비율입니다.

재현율(Recall): 실제 문제 해결 질문 중 모델이 올바르게 문제 해결 질문으로 분류한 비율입니다.

F1 점수(F1 Score): 정밀도와 재현율의 조화 평균으로, 두 지표를 종합적으로 평가할 때 사용됩니다.

새로운 질문의 분류 결과: 사용자가 입력한 새로운 질문에 대한 분류 결과를 얻을 수 있습니다. 이는 모델이 새로운 데이터에 대해 어떻게 작동하는지를 보여줍니다. 이 예시에서는 하나의 새로운 질문을 모델에 제공하고, 모델이 이를 문제 해결 질문으로 분류할지, 아니면 다른 유형의 질문으로 분류할지를 예측합니다.

C_data를 활용하여 추출

In [ ]:
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# 문제 해결과 관련된 키워드 목록 정의
problem_solving_keywords = ['error', 'debugging', 'algorithm', 'optimization']

# 파일 경로 설정 (실제 경로로 변경해야 함)
file_path = '/content/drive/MyDrive/four programming/four programming_Q_list/C_data_Q_list.txt'

# 단일 파일 데이터 로드 및 레이블링
def load_and_label_question(filepath, keywords):
    data = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip().lower()
            label = 'no'
            for keyword in keywords:
                if keyword in line:
                    label = keyword
                    break
            data.append({"question": line, "label": label})
    return pd.DataFrame(data)

# 나이브 베이지안 알고리즘을 사용하여 질문 분류
def classify_questions(data):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(data['question'])
    y = (data['label'] != 'no').astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    classifier = MultinomialNB()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    return classifier, vectorizer

# 엑셀 파일로 저장
def save_to_excel(data, filename='C_data_classified_questions.xlsx'):
    data.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

# 메인 실행 함수
def main():
    data = load_and_label_question(file_path, problem_solving_keywords)
    classifier, vectorizer = classify_questions(data)
    save_to_excel(data)

# 실행
if __name__ == "__main__":
    main()

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       671
           1       0.65      0.52      0.58        79

    accuracy                           0.92       750
   macro avg       0.80      0.74      0.77       750
weighted avg       0.91      0.92      0.92       750

Data saved to C_data_classified_questions.xlsx


java_data를 활용하여 추출

In [ ]:
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# 문제 해결과 관련된 키워드 목록 정의
problem_solving_keywords = ['error', 'debugging', 'algorithm', 'optimization']

# 파일 경로 설정 (실제 경로로 변경해야 함)
file_path = '/content/drive/MyDrive/four programming/four programming_Q_list/java_data_Q_list.txt'

# 단일 파일 데이터 로드 및 레이블링
def load_and_label_question(filepath, keywords):
    data = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip().lower()
            label = 'no'
            for keyword in keywords:
                if keyword in line:
                    label = keyword
                    break
            data.append({"question": line, "label": label})
    return pd.DataFrame(data)

# 나이브 베이지안 알고리즘을 사용하여 질문 분류
def classify_questions(data):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(data['question'])
    y = (data['label'] != 'no').astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    classifier = MultinomialNB()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    return classifier, vectorizer

# 엑셀 파일로 저장
def save_to_excel(data, filename='java_data_classified_questions.xlsx'):
    data.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

# 메인 실행 함수
def main():
    data = load_and_label_question(file_path, problem_solving_keywords)
    classifier, vectorizer = classify_questions(data)
    save_to_excel(data)

# 실행
if __name__ == "__main__":
    main()

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       702
           1       0.49      0.77      0.60        48

    accuracy                           0.93       750
   macro avg       0.74      0.86      0.78       750
weighted avg       0.95      0.93      0.94       750

Data saved to java_data_classified_questions.xlsx


Python_data를 활용하여 추출

In [ ]:
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# 문제 해결과 관련된 키워드 목록 정의
problem_solving_keywords = ['error', 'debugging', 'algorithm', 'optimization']

# 파일 경로 설정 (실제 경로로 변경해야 함)
file_path = '/content/drive/MyDrive/four programming/four programming_Q_list/Python_data_Q_list.txt'

# 단일 파일 데이터 로드 및 레이블링
def load_and_label_question(filepath, keywords):
    data = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip().lower()
            label = 'no'
            for keyword in keywords:
                if keyword in line:
                    label = keyword
                    break
            data.append({"question": line, "label": label})
    return pd.DataFrame(data)

# 나이브 베이지안 알고리즘을 사용하여 질문 분류
def classify_questions(data):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(data['question'])
    y = (data['label'] != 'no').astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    classifier = MultinomialNB()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    return classifier, vectorizer

# 엑셀 파일로 저장
def save_to_excel(data, filename='Python_data_classified_questions.xlsx'):
    data.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

# 메인 실행 함수
def main():
    data = load_and_label_question(file_path, problem_solving_keywords)
    classifier, vectorizer = classify_questions(data)
    save_to_excel(data)

# 실행
if __name__ == "__main__":
    main()

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       661
           1       0.70      0.70      0.70        89

    accuracy                           0.93       750
   macro avg       0.83      0.83      0.83       750
weighted avg       0.93      0.93      0.93       750

Data saved to Python_data_classified_questions.xlsx


Prolog_data를 활용하여 추출

In [ ]:
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# 문제 해결과 관련된 키워드 목록 정의
problem_solving_keywords = ['error', 'debugging', 'algorithm', 'optimization']

# 파일 경로 설정 (실제 경로로 변경해야 함)
file_path =  '/content/drive/MyDrive/four programming/four programming_Q_list/Prolog_data_Q_list.txt'

# 단일 파일 데이터 로드 및 레이블링
def load_and_label_question(filepath, keywords):
    data = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip().lower()
            label = 'no'
            for keyword in keywords:
                if keyword in line:
                    label = keyword
                    break
            data.append({"question": line, "label": label})
    return pd.DataFrame(data)

# 나이브 베이지안 알고리즘을 사용하여 질문 분류
def classify_questions(data):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(data['question'])
    y = (data['label'] != 'no').astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    classifier = MultinomialNB()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    return classifier, vectorizer

# 엑셀 파일로 저장
def save_to_excel(data, filename='Prolog_data_classified_questions.xlsx'):
    data.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

# 메인 실행 함수
def main():
    data = load_and_label_question(file_path, problem_solving_keywords)
    classifier, vectorizer = classify_questions(data)
    save_to_excel(data)

# 실행
if __name__ == "__main__":
    main()

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       711
           1       0.71      0.64      0.68        39

    accuracy                           0.97       750
   macro avg       0.85      0.81      0.83       750
weighted avg       0.97      0.97      0.97       750

Data saved to Prolog_data_classified_questions.xlsx
